In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import json

In [2]:
load_dotenv("../../.env",override=True)
pd.set_option("display.max_columns",500)
import numpy as np

In [3]:
consumerKey = os.environ.get("CONSUMER_KEY")
refreshToken = os.environ.get("TD_REFRESH_TOKEN")
acctID = os.environ.get("TD_ACCT_ID")



In [4]:

response = requests.post("https://api.tdameritrade.com/v1/oauth2/token",
    data = {'grant_type':'refresh_token',
        'refresh_token':refreshToken,
        'client_id':consumerKey,
        'redirect_uri':''}
)

response

<Response [200]>

In [5]:
accessToken = json.loads(response.content)['access_token']

In [6]:

#get positions
url = f'https://api.tdameritrade.com/v1/accounts/{acctID}?fields=positions'
response = requests.get(url,
        headers={'Authorization' : f"Bearer {accessToken}"})



In [7]:
response

<Response [200]>

In [8]:
positionsDict = json.loads(response.content)

In [9]:
balance = positionsDict['securitiesAccount']['currentBalances']['equity']

In [13]:
positions = pd.DataFrame(positionsDict['securitiesAccount']['positions'])
positions['symbol'] = positions['instrument'].apply(lambda x: x['symbol'])
positions = positions[['symbol',
'marketValue',
'longQuantity','averagePrice']].copy()
positions['myWeight'] = 1
positions['equityShare'] = balance*positions['myWeight']/sum(positions['myWeight'])
positions['currentPrice'] = positions['marketValue']/positions['longQuantity']


In [14]:


def weigh_positions(positions: pd.DataFrame,weights: dict[int,list],balance: float) -> pd.DataFrame:
    """_summary_

    Args:
        positions (pd.DataFrame): _description_
        weights (dict[int,list]): _description_
        balance (float): _description_

    Returns:
        pd.DataFrame: _description_
    """
    allWeighted = []
    for k in weights.keys():
        positions['myWeight'] = np.where(positions['symbol'].isin(weights[k]),k,positions['myWeight'])
        allWeighted = allWeighted+weights[k]

    positions['myWeight'] = np.where(~positions['symbol'].isin(allWeighted),1,positions['myWeight'])
    positions['equityShare'] = balance*positions['myWeight']/sum(positions['myWeight'])
    positions['diff']  = positions['equityShare']  - positions['marketValue']
    return positions


In [15]:
weights = {4:['VOO','VONG'],
    3:['AAPL','GOOGL','MSFT','VTV','VGT'],
    2:['AMZN','IMCG','UNH']
    }

positions = weigh_positions(positions,weights,balance)
positions.sort_values(by='diff',ascending=False)


,symbol,marketValue,longQuantity,averagePrice,myWeight,equityShare,currentPrice,diff
17,VTV,2351.27,17.000,140.155813,3,4689.897231,138.310000,2338.627231
30,MSFT,2570.04,11.000,277.850882,3,4689.897231,233.640000,2119.857231
6,UNH,1667.67,3.000,481.746667,2,3126.598154,555.890000,1458.928154
19,VGT,3319.03,10.000,349.163140,3,4689.897231,331.903000,1370.867231
22,ACHR,360.00,125.000,3.875040,1,1563.299077,2.880000,1203.299077
5,CF,527.50,5.000,104.600000,1,1563.299077,105.500000,1035.799077
28,AMGN,539.52,2.000,229.030000,1,1563.299077,269.760000,1023.779077
23,AMD,667.49,11.000,133.497270,1,1563.299077,60.680909,895.809077
18,GOOGL,3804.40,40.000,142.879500,3,4689.897231,95.110000,885.497231
0,VONG,5457.75,95.000,57.537260,4,6253.196308,57.450000,795.446308
